# Politician Activity on Wikipedia by Political Affiliation

The parameters in the cell below can be adjusted to explore other political affiliations and time frames.

### How to explore other political affiliation?
The ***affiliation*** parameter can be use to aggregate politicians by their political affiliations. The column `affiliation` in this [this other notebook](../politicians.ipynb?autorun=true) show the politicians that belong each political affiliation.

***Alternatively***, you can direcly use the [politicians API](http://mediamonitoring.gesis.org/api/politicians/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

In [ ]:
# Parameters: 
affiliation = 'CSU'
aggregation = 'week' # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

## Using API

In [ ]:
base <- "193.175.238.88/api/politicians/"
call1 <- paste(base,"all/", sep="")
get_politicians_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
party_df <- get_politicians_df[get_politicians_df$affiliation == affiliation & !is.null(get_politicians_df$wp_ids) ,]

In [ ]:
base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"
base_ch <- paste(base, "wikipedia/chobs/politicians/page_id/",sep="")
chobs <- data.frame()
wikiwho <- data.frame()
for (wp_id in party_df$wp_ids) {
    for (wp in wp_id) {
        call_chobs <- paste(base_ch, toString(wp), "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
        json_chobs <- fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"))
        if (length(json_chobs$values) != 0)  {
            chobs_unique <- as.data.frame(fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"), flatten = TRUE))
            chobs_unique$wp_user_id = wp
            chobs <- rbind(chobs, chobs_unique)
        }
        call_wikiwho <- paste(base_ww, wp ,"/?start=",from_date,"&end=",to_date,sep="")
        json_wikiwho <- fromJSON(content(GET(call_wikiwho), "text", encoding="UTF-8"))
        wikiwho <- rbind(wikiwho, json_wikiwho$editions)
    }
}

wikiwho$undos <- wikiwho$dels + wikiwho$reins
wikiwho <- summarise(group_by(wikiwho, year_month), undos = sum(undos),revisions = sum(revisions), conflict = sum(conflict), elegibles= sum(elegibles))
wikiwho$conflict_score <- wikiwho$conflict/ wikiwho$elegibles
chobs <- summarise(group_by(chobs, labels), values = sum(values))
wikiwho[is.na(wikiwho)] <- 0

In [ ]:
appended_views = list()
for (wp_title in party_df$wp_titles) {
    for (wp in wp_title) {       
        
        x <- tryCatch(
            {
                call_views <- paste("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/", wp,"/", "daily/",gsub("-", "", from_date),"/",gsub("-", "", to_date), sep="")
                view = as.data.frame(fromJSON(content(GET(call_views), "text", encoding="UTF-8"), flatten = TRUE))
                appended_views <- append(appended_views, list(view))
            },
   error = function(e) e )

    }
}
views  <- dplyr::bind_rows(appended_views)
views<-views[c("items.timestamp", "items.views")]
views$items.timestamp<-as.POSIXct(strptime(views[["items.timestamp"]], format='%Y%m%d%H'),tz = 'GMT')

if (aggregation  == 'day'){
    views<-summarise(group_by(views, time = cut(items.timestamp, "day")), items.views = sum(items.views))
    }
if (aggregation  == 'week'){
    views<-summarise(group_by(views, time = cut(items.timestamp, "week")), items.views = sum(items.views))
    }
if (aggregation  == 'month'){  
    views<-summarise(group_by(views, time = cut(items.timestamp, "month")), items.views = sum(items.views))
    }


In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = chobs, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = values, color="Changes")) + 
  labs(title = "Wikipedia Activity", x = "", y = "Changes") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), legend.position="none") +
  scale_x_date(date_breaks = "1 month")

options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = views, mapping = aes(as.Date(time))) +
  geom_line(aes(y = items.views, color="Views"))  + 
  labs(title = "", x = "Date", y = "Views") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), legend.position="none") +
  scale_x_date(date_breaks = "1 month")

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = wikiwho, mapping = aes(as.Date(year_month))) +
  geom_line(aes(y = undos, color="Undos")) +  
  labs(title = "Wikipedia Disagreements", x = "", y = "Undos") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month") 

options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = wikiwho, mapping = aes(as.Date(year_month))) +  
  geom_line(aes(y = conflict_score, color="Conflict")) +
  labs( x = "Date", y = "Conflict") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month") + expand_limits(y = 0)
